# Energy Forecasting with Deep Learning (Part 1)
author: Andrew Mendez, andrew.mendez@hpe.com

Version: 0.0.1

Date: 3.14.23

This tutorial introduces the process of training a deep learning model for time series forecasting, specifically forecasting energy demand. Time series forecasting is a crucial aspect of many domains, including energy management, where accurate predictions can lead to efficient energy use and help in decision-making processes.

## Dataset
The dataset used in this example contains various energy generation and consumption data. We focus on the 'generation fossil gas' feature to predict future energy demand.

## Model
We employ a Temporal Convolutional Network (TCN) model, known for its effectiveness in handling sequence data like time series.

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from darts import TimeSeries, concatenate
from darts.models import TCNModel
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.metrics import mse, rmse
from darts.datasets import EnergyDataset
import warnings
warnings.filterwarnings('ignore')
import pickle

# Helper function for model configuration
def generate_torch_kwargs():
    return {
        'pl_trainer_kwargs': {
            'accelerator': 'cpu',
        }
    }

## Data Preparation
The following steps prepare the dataset for training our TCN model.

The dataset comprises 4 years of data on electrical consumption, generation, pricing, and weather for Spain. It includes detailed hourly data, allowing for fine-grained analysis and forecasting of energy demand and supply dynamics. More details about the dataset can be seen [here](https://www.kaggle.com/datasets/nicholasjhana/energy-consumption-generation-prices-and-weather)

In [ ]:
# Load the energy dataset

df = pd.read_csv('data/energy_dataset.csv', parse_dates=['time'])
df.set_index('time', inplace=True)

## Time Series Transformation
We transform the dataset to make it suitable for the TCN model. This includes normalizing the data and splitting it into training and validation sets.

We focus on forecasting 'generation hydro run-of-river and poundage', which represents the hydroelectric generation in megawatts (MW). This feature is particularly interesting for forecasting due to its dependence on natural water flow and storage, making it sensitive to weather conditions and potentially more challenging to predict accurately compared to other generation methods.


In [ ]:
def preprocess_train_and_val_and_save(df):
    '''
    '''
    # Aggregating by day to simplify the model's task, aiming to capture daily patterns
    # in hydroelectric generation which might be influenced by daily weather changes
    # and consumption patterns.
    df_day_avg = df.resample('D').mean()
    series = TimeSeries.from_dataframe(df_day_avg, value_cols=['generation hydro run-of-river and poundage'])

    # Splitting the series into training and validation sets, with the split point chosen
    # to ensure the model is trained on a substantial historical dataset while leaving
    # enough recent data for validation.
    train, val = series.split_after(pd.Timestamp('20170901'))

    # Normalize the series to aid the model training process by ensuring numerical values
    # have a mean of 0 and standard deviation of 1. This helps improve the stability and
    # speed of convergence during training.
    scaler = Scaler()
    train_transformed = scaler.fit_transform(train)
    val_transformed = scaler.transform(val)

    # Add day as a covariate:
    # Including the day of the month as a one-hot encoded covariate to provide the model
    # with additional contextual information about potential daily periodicities in
    # hydroelectric generation.
    day_series = datetime_attribute_timeseries(series, attribute='day', one_hot=True)
    return train_transformed, val_transformed, scaler, day_series


train_transformed, val_transformed, scaler, day_series = preprocess_train_and_val_and_save(df)

# Plotting the normalized training and validation series to visualize the data
# the model will be trained and validated on. This visualization helps in understanding
# the general trend and seasonality in the data, providing insights into the model's
# potential forecasting challenges.
plt.figure(figsize=(10, 3))
train_transformed.plot(label='Train')
val_transformed.plot(label='Validation')
plt.legend()
plt.title('Normalized Training and Validation Series for Hydroelectric Generation')
plt.show()

In [ ]:
#save the scaler to use for inferencing
# Assuming 'scaler' is your Scaler object
scaler_file_path = 'scaler.pkl'  # Specify the path to save the scaler

# Save the scaler object to a file
with open(scaler_file_path, 'wb') as file:
    pickle.dump(scaler, file)

## Model Training
We configure and train the TCN model using the prepared dataset.

### Overview of TCN Architecture and Hyperparameters: 
The TCN model uses a series of dilated convolutional layers that allow the network to have an exponentially increasing receptive field. This means the model can incorporate information from points further back in the time series without a proportional increase in computational complexity. The architecture is particularly suited for time series data because it can effectively capture long-term dependencies and patterns. Key features include causal convolutions (ensuring predictions at time t are only dependent on data from times t' <= t) and residual connections (helping with training deep networks by allowing gradients to flow through the network's layers more effectively).

In [ ]:
# Configure and train the TCN model
model = TCNModel(
    # input_chunk_length defines the length of the input sequences. For our model,
    # we're looking at 365 days of data to predict the next 7 days. This parameter
    # is crucial for capturing seasonal patterns within a year.
    input_chunk_length=365,

    # output_chunk_length defines the length of the forecasted output sequence. Here,
    # we aim to predict energy demand for the next 7 days based on the input sequence.
    output_chunk_length=7,

    # n_epochs specifies the number of times the model will work through the entire
    # training dataset. More epochs can lead to better learning, but also a risk of overfitting.
    n_epochs=50,

    # dropout is a regularization technique where input and recurrent connections to
    # LSTM units are probabilistically excluded from activation and weight updates while
    # training a network. This helps prevent overfitting. We set it to 0.2, meaning there's
    # a 20% chance that any given connection is dropped.
    dropout=0.2,

    # dilation_base controls the spacing between the kernel points in dilated convolutions.
    # It's part of how TCNs manage to have a large receptive field while keeping the model
    # depth manageable. A dilation_base of 2 means the spacing between kernel points
    # will double with each layer, helping the model to efficiently learn hierarchical
    # representations of time series data.
    dilation_base=2,
    weight_norm=True,
    save_checkpoints=True,
    nr_epochs_val_period=1,
    force_reset=True,
    # kernel_size is the size of the convolutional kernel. It determines how many input
    # values will be considered at once by each convolution operation. A size of 5 means
    # each convolution will combine information from 5 time steps.
    kernel_size=5,

    # num_filters specifies the number of filters in the convolutional layers, which
    # translates to the number of output channels. More filters can capture more information
    # but increase computational complexity. We choose 8 as a balance.
    num_filters=8,

    # random_state sets the seed for random number generation, ensuring reproducibility
    # of the model training process.
    random_state=0,

    # Additional arguments passed to the PyTorch Lightning trainer. In this case, we're
    # specifying the training to be performed on CPU and customizing callbacks for training
    # progress. This is handled by the generate_torch_kwargs() helper function.
    **generate_torch_kwargs()
)


# Fit the model
model.fit(
    series=train_transformed,
    past_covariates=day_series,
    val_series=val_transformed,
    val_past_covariates=day_series
)
# Save the model
model.save('models/TCN_model.pt')

## Model Evaluation
After training, we evaluate the model's performance on the validation set using backtesting.

Run backtest validation: Historical forecasting (backtesting) simulates the prediction of past data at various points in time, as if we were predicting the future. Here, we're using our trained model to forecast 7 days into the future, given past covariates, and we're not retraining the model in the process. This allows us to compare the model's forecasts to the actual observed values in our validation set.

In [ ]:
backtest = model.historical_forecasts(
    series=val_transformed,
    past_covariates=day_series,
    forecast_horizon=7,
    retrain=False,
    verbose=True
)

# Plotting actual vs forecasted
plt.figure(figsize=(10, 6))
val_transformed.plot(label='Actual')
backtest.plot(label='Forecast')
plt.legend()
plt.title('Validation Set - Actual vs Forecast')
plt.show()

Comparing the backtest forecasts against the ground truth in the validation set to quantify the model's performance. Common metrics for this purpose include RMSE (Root Mean Square Error) and MAE (Mean Absolute Error). Lower values indicate better model performance.

The **RMSE** metric provides the standard deviation of the residuals (prediction errors), indicating how far, on average, the forecasts are from the actual values. **MAE** offers a straightforward average of absolute errors. Both metrics are useful for understanding the accuracy and performance of the forecasting model in real-world terms.

In [ ]:
from darts.metrics import rmse, mae

# Calculate RMSE and MAE
rmse_val = rmse(val_transformed, backtest)
mae_val = mae(val_transformed, backtest)

# Display the metrics
print(f'RMSE (Validation): {rmse_val:.2f}')
print(f'MAE (Validation): {mae_val:.2f}')

## Summary
This tutorial demonstrated the process of forecasting energy demand using a TCN model. Starting from data preparation to model training and evaluation, each step was detailed for clarity. The forecasted results on the validation set show the model's ability to predict future values, essential for decision-making in energy management.